# Метапрограммирование в Python

## Задача 1 Применение метаклассов

In [5]:
class AttrLoggingMeta(type):
    def __new__(cls, name, bases, dct):
        # Оборачивание всех методов и атрибутов для логирования
        for attr_name, attr_value in dct.items():
            if callable(attr_value):
                dct[attr_name] = cls.wrap_method(attr_name, attr_value)
        
        # Создание класса через базовый метакласс
        new_class = super().__new__(cls, name, bases, dct)
        return new_class

    @staticmethod
    def wrap_method(name, method):
        def wrapped_method(self, *args, **kwargs):
            print(f"Вызов метода {name}")
            return method(self, *args, **kwargs)
        return wrapped_method

    def __call__(cls, *args, **kwargs):
        instance = super().__call__(*args, **kwargs)
        for name, value in cls.__dict__.items():
            if isinstance(value, property):
                cls.wrap_property(instance, name)
        return instance

    @staticmethod
    def wrap_property(instance, name):
        prop = getattr(type(instance), name)

        def getter(self):
            value = prop.fget(self)
            type(self).log_read(name, value, self)
            return value

        def setter(self, value):
            type(self).log_write(name, value, self)
            prop.fset(self, value)

        setattr(type(instance), name, property(getter, setter))

    @staticmethod
    def log_access(name, value):
        print(f"Доступ к атрибуту {name} со значением {value}")

    @staticmethod
    def log_read(name, value, instance):
        print(f"Чтение атрибута {name}: {value}")

    @staticmethod
    def log_write(name, value, instance):
        print(f"Запись атрибута {name} со значением {value}")

class LoggedClass(metaclass=AttrLoggingMeta):
    def __init__(self):
        self._custom_method = 42

    @property
    def custom_method(self):
        return self._custom_method

    @custom_method.setter
    def custom_method(self, value):
        self._custom_method = value

    def other_custom_method(self):
        print("Выполнение other_custom_method")


In [6]:
if __name__ == "__main__":
    instance = LoggedClass()
    print(instance.custom_method)  # Доступ к атрибуту
    instance.custom_method = 78   # Изменение атрибута
    instance.other_custom_method()  # Вызов метода

Вызов метода __init__
Чтение атрибута custom_method: 42
42
Запись атрибута custom_method со значением 78
Вызов метода other_custom_method
Выполнение other_custom_method


## Задача 2 Динамическое создание класса

In [2]:

def create_class_with_methods(class_name, attributes, methods):
    return type(class_name, (object,), {**attributes, **methods})


In [3]:
if __name__ == "__main__":
    # Задача 2: динамическое создание класса
    attributes = {'species': 'Human', 'age': 25}
    methods = {
        'greet': lambda self: f"Здравствуйте, я {self.species} и мне {self.age} годков."
    }
    DynamicClass = create_class_with_methods('DynamicClass', attributes, methods)
    dynamic_instance = DynamicClass()
    print(dynamic_instance.greet())


Здравствуйте, я Human и мне 25 годков.


In [9]:
def create_class_with_methods(class_name, attributes, methods):
    # Создаём класс с аттрибутами и методами
    new_class = type(class_name, (object,), {**attributes, **methods})
    
    # Создаём экземпляр класса
    instance = new_class()

    # Добавляем атрибуты непосредственно к экземпляру
    for attr, value in attributes.items():
        setattr(instance, attr, value)

    return new_class, instance

if __name__ == "__main__":
    # Задача 2: динамическое создание класса
    attributes = {'species': 'МИША', 'age': 45}
    methods = {
        'greet': lambda self: f"Здравствуйте, я {self.species} и мне {self.age} годков."
    }
    
    # Создание класса и экземпляра
    DynamicClass, dynamic_instance = create_class_with_methods('DynamicClass', attributes, methods)
    
    # Демонстрация работы с методом
    print(dynamic_instance.greet())

    # Демонстрация динамических изменений
    # Добавление нового атрибута
    setattr(dynamic_instance, 'name', 'Вася')
    print(f"Новый аттрибут добавлен name = {getattr(dynamic_instance, 'name')}")

    # Изменение существующего атрибута
    setattr(dynamic_instance, 'age', 35)
    print(f"Изменение атрибута age = {getattr(dynamic_instance, 'age')}")
  
    # Удаление атрибута
    delattr(dynamic_instance, 'species')
    try:
        print(getattr(dynamic_instance, 'species'))
    except AttributeError:
        print("Attribute 'species' has been deleted.")

    # Добавление нового метода
    setattr(dynamic_instance, 'say_goodbye', lambda: "Пока!")
    print(dynamic_instance.say_goodbye())


Здравствуйте, я МИША и мне 45 годков.
Новый аттрибут добавлен name = Вася
Изменение атрибута age = 35
МИША
Пока!


## Задача 3 Генерация кода

In [23]:
def generate_complex_function(function_name, parameters, function_body):  
    # Создаём строку с определением функции  
    func_definition = f"def {function_name}({', '.join(parameters)}):\n"  
    func_definition += f"    {function_body.replace('\n', '\n    ')}"  # Добавляем отступы  
    
    # Выполняем определение функции  
    exec(func_definition, globals())  
    return eval(function_name)  

if __name__ == "__main__":  
    function_name = 'complex_function'  
    parameters = ['x', 'y']  
    function_body = """  
if x > y:  
    return x - y  
else:  
    return y - x  
"""  
    
    complex_func = generate_complex_function(function_name, parameters, function_body)  
    print(complex_func(10, 5))  # 5  
    print(complex_func(5, 10))   # 5

5
5


In [21]:
def generate_complex_function(function_name, parameters, function_body):
    # Создание строки с кодом для функции
    params_str = ', '.join(parameters)
    
    # Добавление отступов в тело функции
    indented_body = "\n    ".join(function_body.split("\n"))
    function_code = f"""
def {function_name}({params_str}):
    {indented_body}
"""
    
    # Выполнение сгенерированного кода с помощью exec
    exec(function_code)
    
    # Возвращаем ссылку на сгенерированную функцию
    return globals()[function_name]

if __name__ == "__main__":
    # Пример использования
    function_name = 'complex_function'
    parameters = ['x', 'y']
    function_body = """
if x > y:
    return x - y
else:
    return y - x
"""
    
    # Генерация функции
    complex_func = generate_complex_function(function_name, parameters, function_body)
    
    # Вызов сгенерированной функции
    print(complex_func(10, 5))  # Output: 5
    print(complex_func(5, 10))  # Output: 5


5
5
